## Preferencias Británicas
Se consideran los siguientes vectores de atributos binarios:
*(scones, cerveza, whisky, avena, futbol)*

In [132]:
import pandas as pd

df = pd.read_excel('../../res/tp1/PreferenciasBritanicos.xlsx', sheet_name='Hoja1')

row_count = df.shape[0]
# Get number of records from each nationality
nationality_count = df['Nacionalidad'].value_counts()
nationalities = nationality_count.keys()
# Get relative frequency for each nationality (ie: P(v))
relative_freq = nationality_count.apply(lambda x: x / row_count)
# Get conditional probabilities for each attribute, given a nationality (ie: P(a_i | v))
cond_probs = df.groupby('Nacionalidad').apply(lambda x: x.sum() / len(x))

'''
Given a tuple with preferences (ie: (1,1,1,0,1)), returns the most likely nationality
'''
def classify(prefs):
    preferences = pd.DataFrame(data=(prefs,), columns=('scones', 'cerveza', 'wiskey', 'avena', 'futbol'))

    vs = nationality_count.astype(dtype=float, copy=True)
    for nationality in nationalities:
        vs[nationality] = preferences.apply(lambda x: get_cond_prob_by_preference(x, nationality) * relative_freq[nationality]).prod()

    vnb = vs.idxmax(axis=1)
    
    return vnb
    
def get_cond_prob_by_preference(preference, nationality):
    cond_prob = cond_probs.loc[nationality, preference.name]
    return cond_prob if preference.values[0] == 1 else (1 - cond_prob)


Se clasificó el ejemplo x1 = (1, 0, 1, 1, 0) determinando si correspondía a las preferencias de una persona inglesa o escosesa:

In [135]:
print(f"La persona es {'Escocesa' if classify((1,0,1,1,0)) == 'E' else 'Inglesa'}")

La persona es Escocesa


Se clasificó el ejemplo x2 = (0, 1, 1, 0, 1) determinando si correspondía a las preferencias de una persona inglesa o escosesa:

In [136]:
print(f"La persona es {'Escocesa' if classify((0,1,1,0,1)) == 'E' else 'Inglesa'}")

La persona es Inglesa
